In [1]:
import json
from glob import glob
import fnmatch
from multiprocessing import Pool
from pathlib import Path
import os, itertools
from typing import *
from tqdm.notebook import tqdm
from importlib import reload
from pickle import dumps, loads, dump, load

from stexls import *

In [3]:
root = Path.home()/'MathHub'
cache = root / 'stexls-cache.bin'

In [5]:
with open(cache, 'rb') as fd:
    linker = load(fd)

In [23]:
for o, l in linker.links.items():
    if l.errors:
        for loc, errs in l.errors.items():
            try:
                loc.path
            except:
                raise


AttributeError: 'PosixPath' object has no attribute 'decode'

In [27]:
print(loc)

errs

[Location uri="/home/marian/MathHub/smglom/meta-inf/lib/pre.fi.tex" range=[Range (0 0) (6 -1)]]


[stexls.util.latex.exceptions.LatexException('Broken parser stack: [[Node "\\documentclass[mh]{smglom}\n\\input{localpaths}\n\\usepackage[utf8]{inputenc}\n\\usepackage[finnish]{babel}\n\\libinput{preamble}\n\\begin{document}"], [Environment name=[Node "{document}"]]]')]